In [ ]:
import itertools
from math import *

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from numpy.linalg import inv

from utils.colormap import truncate_colormap
from utils import itertools_recipes

from ml.gradient_descent import GradientDescentOptimizer

In [ ]:
reds = np.random.multivariate_normal([5, 3], [[2, -1], [-1, 1]], 80)
blues = np.random.multivariate_normal([1, 0.5], [[5, 2], [2, 2]], 50)

In [ ]:
RS = np.c_[(np.ones((reds.shape[0], 1)), reds)]
BS = np.c_[(np.ones((blues.shape[0], 1)), blues)]
XS = np.r_[RS, BS]
YS = np.r_[np.zeros((reds.shape[0], 1)), np.ones((blues.shape[0], 1))]
M = XS.shape[0]

In [ ]:
def plotData():
    plt.scatter(*reds.T, c='r',  linewidths=0.5, edgecolors='black')
    plt.scatter(*blues.T, c='b',  linewidths=0.5, edgecolors='black');
    
plotData()

In [ ]:
h = lambda θ: lambda x: 1.0 / (1.0 + exp(-np.dot(θ, x)))

In [ ]:
# red  => y=0
# blue => y=1

def J(θ):
    h_θ = h(θ)
    redJ = sum(log(1 - h_θ(red)) for red in RS)
    blueJ = sum(log(h_θ(blue)) for blue in BS)
    return -(redJ + blueJ) / M

In [ ]:
def dJ(θ):
    h_θ = h(θ)
    return 1 / M * sum(x * (h_θ(x) - y) for x, y in zip(XS, YS))

In [ ]:
optimizer = GradientDescentOptimizer(J=J, δJ=dJ, α=0.7, start_θ=[0, 0, 0])

In [ ]:
θ_opt = optimizer.nth(4000)

In [ ]:
θ_opt

In [ ]:
J([5, -1, -1]) # My guess by hand

In [ ]:
J(θ_opt)

In [ ]:
xmin = -3
xmax = 10
ymin = -3
ymax = 6

def plotHypothesis(θ):
    z_func = np.vectorize(lambda x1, x2: h(θ)(np.array([1, x1, x2])))
    Z = z_func(np.linspace(xmin, xmax, 50)[:, None], np.linspace(ymin, ymax, 50))

    plt.figure(figsize=(15,6))
    plt.imshow(
        Z.T,
        extent=(xmin, xmax, ymin, ymax),
        interpolation='none',
        cmap=truncate_colormap("PuOr", 0.3, 0.7),
        origin='lower'
    )
    plt.colorbar()
    plotData()
    plt.ylim(ymin, ymax)
    plt.xlim(xmin, xmax)

In [ ]:
plt.figure(1)
plotHypothesis(optimizer.nth(1))
plt.title("After 1st Iteration")
plt.figure(2)
plotHypothesis(optimizer.nth(40))
plt.title("After 40 Iterations")
plt.figure(3)
plotHypothesis(optimizer.nth(500))
plt.title("After 500 Iterations")
plt.figure(4)
plotHypothesis(θ_opt)
plt.title("After 4000 Iterations");

In [ ]:
optimizer.plot_progress(100, α=1)
optimizer.plot_progress(100, α=0.7)
optimizer.plot_progress(100, α=0.5)
optimizer.plot_progress(100, α=0.1)
optimizer.plot_progress(100, α=0.05)